eliminar outliers
graficar

In [26]:
import pandas as pd
import warnings

In [31]:
df = pd.read_excel('C:/Python/Data Science/Proyecto_2/Datasets/Internet.xlsx', sheet_name=5)
print(df.dtypes)
df

Año              int64
Trimestre        int64
ADSL             int64
Cablemodem       int64
Fibra óptica     int64
Wireless         int64
Otros            int64
Total            int64
Periodo         object
dtype: object


,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2024,2,733491,5867504,4169958,593197,191957,11556107,Abr-Jun 2024
1,2024,1,774475,5986957,4015101,598682,257941,11633156,Ene-Mar 2024
2,2023,4,836390,6022532,3908183,585760,194796,11547661,Oct-Dic 2023
3,2023,3,897895,6018832,3708718,581436,200027,11406908,Jul-Sept 2023
4,2023,2,1006509,5997149,3463988,581823,202428,11251897,Abr-Jun 2023
5,2023,1,1204417,6002582,3243411,576647,214838,11241895,Ene-Mar 2023
6,2022,4,1315373,6037457,3060575,572969,221740,11208114,Oct-Dic 2022
7,2022,3,1395277,6031970,2871541,557110,235230,11091128,Jul-Sept 2022
8,2022,2,1468333,5979214,2723285,556243,231609,10958684,Abr-Jun 2022
9,2022,1,1533240,6073426,2219533,545814,251996,10624009,Ene-Mar 2022


In [21]:
df_duplicados = df[df.duplicated()]
df_duplicados

,Partido,Localidad,link Indec,Velocidad (Mbps),Provincia,Accesos


In [24]:
df.isna().sum()

Partido             0
Localidad           0
link Indec          1
Velocidad (Mbps)    0
Provincia           0
Accesos             0
dtype: int64

In [25]:
df.duplicated().sum()

np.int64(0)

In [23]:
df_na = df.dropna()
df_na

,Partido,Localidad,link Indec,Velocidad (Mbps),Provincia,Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.0,1.0
1,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.5,2.0
2,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.75,19.0
3,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.0,85.0
4,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.5,145.0
...,...,...,...,...,...,...
18859,TUCUMAN,Yerba Buena,Yerba Buena - Marcos Paz,90119030,100.0,5779.0
18860,TUCUMAN,Yerba Buena,Yerba Buena - Marcos Paz,90119030,300.0,4570.0
18861,TUCUMAN,Yerba Buena,Yerba Buena - Marcos Paz,90119030,500.0,442.0
18862,TUCUMAN,Yerba Buena,Yerba Buena - Marcos Paz,90119030,600.0,223.0
